In [1]:
import pandas as pd

from src.dataset import process_dataset

df = pd.read_csv('../res/dataset.csv')
df

,hosp_0,diag_0,dead_0,hosp_1,diag_1,dead_1,hosp_2,diag_2,dead_2,hosp_3,...,diag_39,dead_39,hosp_40,diag_40,dead_40,hosp_41,diag_41,dead_41,init_zone,actuated_zone
0,6.0,350.0,85.0,5.0,355.0,85.0,4.0,359.0,85.0,4.0,...,426.0,91.0,2.0,427.0,91.0,2.0,427.0,91.0,W,R
1,6.0,407.0,85.0,7.0,410.0,85.0,5.0,413.0,86.0,6.0,...,436.0,91.0,1.0,437.0,91.0,1.0,438.0,91.0,R,Y
2,2.0,427.0,91.0,2.0,427.0,91.0,2.0,427.0,91.0,1.0,...,461.0,92.0,3.0,465.0,92.0,3.0,466.0,92.0,Y,O
3,2.0,440.0,91.0,2.0,441.0,91.0,3.0,442.0,91.0,4.0,...,482.0,92.0,2.0,484.0,92.0,2.0,485.0,92.0,O,O
4,3.0,467.0,92.0,2.0,468.0,92.0,2.0,470.0,92.0,1.0,...,517.0,93.0,2.0,519.0,93.0,2.0,521.0,93.0,O,W
5,3.0,485.0,92.0,3.0,486.0,92.0,3.0,486.0,92.0,4.0,...,551.0,95.0,3.0,552.0,96.0,3.0,553.0,96.0,W,Y
6,2.0,521.0,93.0,2.0,523.0,93.0,1.0,525.0,94.0,1.0,...,568.0,97.0,6.0,571.0,98.0,6.0,573.0,98.0,Y,Y
7,7.0,328.0,73.0,6.0,331.0,73.0,6.0,337.0,73.0,7.0,...,361.0,75.0,4.0,362.0,75.0,5.0,363.0,75.0,R,W
8,2.0,343.0,75.0,1.0,343.0,75.0,0.0,343.0,75.0,0.0,...,377.0,79.0,2.0,377.0,79.0,2.0,377.0,79.0,W,O
9,6.0,363.0,75.0,6.0,364.0,75.0,6.0,364.0,76.0,4.0,...,400.0,79.0,1.0,401.0,79.0,1.0,402.0,79.0,O,Y


In [3]:
x, y = process_dataset(df)
x.shape, y.shape

((14, 53), (14, 2))